In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing
import tensorflow_datasets as tfds

2024-05-15 03:05:36.723228: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 03:05:36.723342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 03:05:36.888239: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
max_len = 200
n_words = 10000
dim_embedding = 256
EPOCHS = 20
BATCH_SIZE = 500

In [3]:
def load_data():
    (X_train,y_train),(X_test,y_test) = datasets.imdb.load_data(num_words=n_words)
    X_train = preprocessing.sequence.pad_sequences(X_train,maxlen=max_len)
    X_test = preprocessing.sequence.pad_sequences(X_test,maxlen=max_len)
    return (X_train,y_train),(X_test,y_test)

In [4]:
def build_model():
    model = models.Sequential()
    # Input :- Embedding Layer.
    # The model will take as input an integer matrix of size (batch, input_length).
    # The model will output dimension (input_length,dim_embedding).
    # The largest integer in the input should be no larger than 
    # n_words (vocabulary size)
    model.add(layers.Embedding(n_words,dim_embedding,
                              input_length=max_len))
    model.add(layers.Dropout(0.3))
    # Takes the maximum value of either feature vector from each of
    # the n_words features.
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1,activation='sigmoid'))
    return model

In [5]:
(X_train,y_train),(X_test,y_test) = load_data()
model = build_model()
model.summary()

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(optimizer="adam",loss="binary_crossentropy",
             metrics=["accuracy"])
score = model.fit(X_train,y_train,epochs=EPOCHS,batch_size=BATCH_SIZE,
                 validation_data=(X_test,y_test))

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 389ms/step - accuracy: 0.5841 - loss: 0.6771 - val_accuracy: 0.8171 - val_loss: 0.5466
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 379ms/step - accuracy: 0.8279 - loss: 0.4470 - val_accuracy: 0.8527 - val_loss: 0.3559
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 377ms/step - accuracy: 0.8797 - loss: 0.2975 - val_accuracy: 0.8700 - val_loss: 0.3103
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 386ms/step - accuracy: 0.9111 - loss: 0.2289 - val_accuracy: 0.8741 - val_loss: 0.2949
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 384ms/step - accuracy: 0.9329 - loss: 0.1815 - val_accuracy: 0.8737 - val_loss: 0.2946
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 390ms/step - accuracy: 0.9527 - loss: 0.1382 - val_accuracy: 0.8720 - val_loss: 0.2957
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 394ms/step - accuracy: 0.9673 - loss: 0.1064 - val_accuracy: 0.8706 - val_loss: 0.3029
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 383ms/step - accuracy: 0.9787 - loss: 0.0812 - val_accu

In [7]:
score = model.evaluate(X_test,y_test,batch_size=BATCH_SIZE)
print("\nTest score:",score[0])
print("Test accuracy:",score[1])

50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.8508 - loss: 0.4955

Test score: 0.5059247612953186
Test accuracy: 0.8479200005531311
